In [111]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import undetected_chromedriver as uc
from selenium_stealth import stealth
from selenium.webdriver.common.by import By

In [112]:
HOME_ADDRESS = "https://historicalnewspapers.lib.purdue.edu/?a=cl&cl=CL1&sp=PE&e=-------en-20--1--txt-txIN--------1"
options = Options()
#options.add_argument("--headless")
driver = uc.Chrome(options=options)
stealth(driver,
                languages=["en-US", "en"],
                vendor="Google Inc.",
                platform="Win32",
                webgl_vendor="Intel Inc.",
                renderer="Intel Iris OpenGL Engine",
                fix_hairline=True
        )

driver.implicitly_wait(5)
driver.get(HOME_ADDRESS)
input("Enter anything when captcha complete")
print(driver.title)

The Purdue Exponent (1889-2017) — Browse by title — Purdue University Newspapers Collection


In [98]:
start_year = 1936

In [99]:
calendar = driver.find_element(By.ID, 'datebrowserrichardtoplevelcalendar')

In [100]:
year = driver.find_element(By.XPATH, f"//h2[contains(text(), '{start_year}')]/following-sibling::div[1]")

In [101]:
first_link = year.find_element(
    By.XPATH,
    ".//ul[contains(@class,'nav') and contains(@class,'nav-pills')]//a[@class='nav-link'][1]"
)

In [102]:
first_link = first_link.get_attribute("href")

In [103]:
driver.get(first_link)

In [104]:
super_links = []

In [113]:
import json
from IPython.display import clear_output

num_years = 15
num_months = 12 * num_years
links_filename = f'/home/malinkyzubr/Desktop/HIST302/302hist/files/links{start_year}-{num_months}.json'


try:
    with open(links_filename, 'r') as f:
        super_links = json.load(f)
except:
    for month in range(num_months + 1):
        try:
            calendar_box = driver.find_element(By.ID, "datebrowserrichardmonthlevelcalendar")
            weeks = calendar_box.find_elements(By.XPATH, "./*[position() > 1]")
            days = calendar_box.find_elements(
                By.XPATH,
                ".//div[@class='datebrowserrichardmonthlevelcalendardaycell' and not(contains(@class, ' '))]"
            )

            for day in days:
                date = day.find_element(By.CSS_SELECTOR, ".hiddenwhennotsmall")
                link = day.find_element(By.CSS_SELECTOR, "li.list-group-item a")
                href = link.get_attribute("href")
                super_links.append((date.get_attribute("innerHTML"), href))
                print(f"Links Extracted: {len(super_links)}")
        except:
            pass
        next_month = driver.find_element(
            By.XPATH,
            "//a[span[text()='Next month']]"
        )
        href = next_month.get_attribute("href")
        driver.get(href)
    with open(links_filename, 'w') as f:
        json.dump(super_links, f)


In [114]:
import os

files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
files = [file.replace(".txt", "") for file in files]

In [115]:
def text_get():
    container = driver.find_element(By.ID, "documentdisplayleftpanesectiontextcontainer")

    paragraphs = container.find_elements(
        By.CSS_SELECTOR,
        "p[dir='auto']"
    )

    text = "\n".join([p.text for p in paragraphs])

    driver.get(super_link)

    return text

In [116]:
super_links = [link for link in super_links if link[0] not in files]

In [ ]:
import queue
# takes a minute to correct after internal error

retry_queue: queue.Queue[tuple[str, str]] = queue.Queue(maxsize=14)

files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
files = [file.replace(".txt", "") for file in files]
super_links = [link for link in super_links if link[0] not in files]
while len(super_links) > 0:
    if retry_queue.full():
        date, link = retry_queue.get()
    else:
        date, link = super_links.pop(0)
    driver.get(link)

    agg_text = ""
    container = driver.find_element(By.ID, "documentdisplayleftpanecontentscontainer")
    links = container.find_elements(By.CSS_SELECTOR, "a.pagetocnodelink")
    if not links:
        retry_queue.put((date, link))
    else:
        links = [link.get_attribute("href") for link in links]

        for link in links:
            driver.get(link)
            agg_text += text_get()
        with open(f"/home/malinkyzubr/Desktop/HIST302/302hist/files/{date}.txt", "w") as f:
            f.write(agg_text)
        files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
        files = [file.replace(".txt", "") for file in files]

In [108]:
files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
files = [file.replace(".txt", "") for file in files]
iteration = 0
while len(files) < len(super_links):
    counter = 0
    failures = 0
    start = time.time()
    for date, super_link in super_links:
        if date in files:
            continue
        driver.get(super_link)

        agg_text = ""
        container = driver.find_element(By.ID, "documentdisplayleftpanecontentscontainer")
        links = container.find_elements(By.CSS_SELECTOR, "a.pagetocnodelink")
        if not links:
            failures += 1
            continue
        links = [link.get_attribute("href") for link in links]

        for link in links:
            driver.get(link)
            agg_text += text_get()
        if not agg_text:
            failures += 1
            continue
        with open(f"/home/malinkyzubr/Desktop/HIST302/302hist/files/{date}.txt", "w") as f:
            f.write(agg_text)
        counter += 1
    print(f"ITERATION{iteration}: {counter} files downloaded in {(time.time() - start) / 60} minutes with {failures} failures.")
    files = os.listdir("/home/malinkyzubr/Desktop/HIST302/302hist/files")
    files = [file.replace(".txt", "") for file in files]
    iteration += 1

ITERATION0: 0 files downloaded in 0.08820226987202963 minutes with 1 failures.
ITERATION1: 0 files downloaded in 0.08817911942799886 minutes with 1 failures.
ITERATION2: 2 files downloaded in 0.3343983173370361 minutes with 1 failures.
ITERATION3: 0 files downloaded in 0.08830267190933228 minutes with 1 failures.
ITERATION4: 0 files downloaded in 0.08743164539337159 minutes with 1 failures.
ITERATION5: 0 files downloaded in 0.08809139331181844 minutes with 1 failures.
ITERATION6: 0 files downloaded in 0.08806768655776978 minutes with 1 failures.
ITERATION7: 0 files downloaded in 0.08830888668696085 minutes with 1 failures.
ITERATION8: 0 files downloaded in 0.0877571702003479 minutes with 1 failures.
ITERATION9: 0 files downloaded in 0.08809855779012045 minutes with 1 failures.
ITERATION10: 0 files downloaded in 0.08812124729156494 minutes with 1 failures.
ITERATION11: 0 files downloaded in 0.0878430445988973 minutes with 1 failures.
ITERATION12: 1 files downloaded in 0.1839462439219157

KeyboardInterrupt: 